# Learning and Decision Making

## Laboratory 6: Reinforcement learning

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. The windy gridworld domain

Consider the larger version of the windy gridworld domain depicted in the figure below.

<img src="windy.png" width="400px">

In it, a boat must navigate a 7 &times; 10 gridworld, to reach the goal cell, marked with _G_. There is a crosswind upward through the middle of the grid, in the direction indicated by the gray arrows. The boat has available the standard four actions -- _Up_, _Down_, _Left_ and _Right_. In the region affected by the wind, however, the resulting next state is shifted upward as a consequence of the crosswind, the strength of which varies from column to column. The strength of the wind is given below each column, and corresponds to the number of cells that the movement is shifted upward. For example, if the boat is one cell to the right of the goal, then the action _Left_ takes you to the cell just above the goal.

The agent pays a cost of 1 in every step before reaching the goal. The problem can be described as an MDP $(\mathcal{X},\mathcal{A},\mathbf{P},c,\gamma)$ as follows.

In [124]:
%matplotlib notebook
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

np.set_printoptions(threshold=10)

# Problem specific parameters
WIND = (0, 0, 0, 1, 1, 1, 2, 2, 1, 0)
nrows = 7
ncols = 10
init = [3, 0]
goal = [3, 7]

# States
X = [[x, y] for x in range(nrows) for y in range(ncols)]
nX = len(X)

# Actions
A = ['U', 'D', 'L', 'R']
nA = len(A)

# Transition probabilities
P = dict()
P['U'] = np.zeros((nX, nX))
P['D'] = np.zeros((nX, nX))
P['L'] = np.zeros((nX, nX))
P['R'] = np.zeros((nX, nX))

for i in range(len(X)):
    x = X[i]
    y = dict()
    
    y['U'] = [x[0] - WIND[x[1]] - 1, x[1]]
    y['D'] = [x[0] - WIND[x[1]] + 1, x[1]]
    y['L'] = [x[0] - WIND[x[1]], x[1] - 1]
    y['R'] = [x[0] - WIND[x[1]], x[1] + 1]
    
    for k in y:
        y[k][0] = max(min(y[k][0], nrows - 1), 0)
        y[k][1] = max(min(y[k][1], ncols - 1), 0)
        j = X.index(y[k])
        P[k][i, j] = 1

c = np.ones((nX, nA))
c[X.index(goal), :] = 0

gamma = 0.99

# -- Pretty print

print('\n- MDP problem specification: -\n')

print('States:')
print(np.array(X))

print('\nActions:')
print(A)

print('\nTransition probabilities:')
for a in A:
    print('Action', a)
    print(P[a])
    
print('\ncost:')
print(c)

print('\nStart state:', init)
print('\nGoal state:', goal)


- MDP problem specification: -

States:
[[0 0]
 [0 1]
 [0 2]
 ..., 
 [6 7]
 [6 8]
 [6 9]]

Actions:
['U', 'D', 'L', 'R']

Transition probabilities:
Action U
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Action D
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
Action L
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Action R
[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]

cost:
[[ 1.  1

---

#### Activity 1.        

Compute the optimal _Q_-function for the MDP defined above using value iteration. As your stopping condition, use an error between iterations smaller than `1e-8`.

---

In [125]:
def value_iteration():
    i = 0
    err = 1
    J = np.zeros((len(X)))
    Q = np.zeros((len(X), len(A)))
    while err > 1e-8:
        Qup = c[:,0] + np.multiply(gamma, np.dot(P['U'], J))        
        Qdown = c[:,1] + np.multiply(gamma, np.dot(P['D'], J))
        Qleft = c[:,2] + np.multiply(gamma, np.dot(P['L'], J))
        Qright = c[:,3] + np.multiply(gamma, np.dot(P['R'], J))
        
        Jnew = np.min((Qup, Qdown, Qleft, Qright), axis = 0)
        err = np.linalg.norm(Jnew - J)
        i +=1
        J = Jnew
    Q[:,0] = Qup
    Q[:,1] = Qdown
    Q[:,2] = Qleft
    Q[:,3] = Qright


    return Q

Q = value_iteration()
print(Q)

[[ 88.97864977  88.97864977  88.97864977  88.867323  ]
 [ 88.867323    88.867323    88.97864977  88.75487172]
 [ 88.75487172  88.75487172  88.867323    88.64128457]
 ..., 
 [ 87.18534603  87.44035764  88.29359257  87.31349257]
 [ 87.31349257  87.44035764  87.44035764  87.56595406]
 [ 87.56595406  87.56595406  87.44035764  87.56595406]]


---

#### Activity 2.        

Write down a Python function that, given a Q-function $Q$ and a state $x$, selects a random action using the $\epsilon$-greedy policy obtained from $Q$ for state $x$. Your function should receive an optional parameter, corresponding to $\epsilon$, with default value of 0.1. 

**Note:** In the case of two actions with the same value, your $\epsilon$-greedy policy should randomize between the two.

---

In [126]:
def epsilon_choice(Q, x, eps=0.1):
    actions = [0, 1, 2, 3]
    random = np.random.rand()
    
    state_i = X.index(x)
    Qvals = Q[state_i]
    
    if(random < eps):
        return np.random.choice(actions)
    
    else:
        return actions[np.argmin(Qvals)]
    
    
'''probabilities = np.zeros((nA))
state_index = X.index(x)
Qvals = Q[state_index]
minQ = np.min(Qvals)
min_indexes = []
for i in range(len(Qvals)):
    if Qvals[i] == minQ:
        min_indexes += [i]

if(len(min_indexes) == nA):
    return np.random.choice(actions)
else:  
    for i in range(len(Qvals)):
        if i in min_indexes:
            probabilities[i] = (1 - eps) / len(min_indexes)
        else:
            probabilities[i] = eps / (nA - len(min_indexes))
    return np.random.choice(actions, p=probabilities)
   '''     


'probabilities = np.zeros((nA))\nstate_index = X.index(x)\nQvals = Q[state_index]\nminQ = np.min(Qvals)\nmin_indexes = []\nfor i in range(len(Qvals)):\n    if Qvals[i] == minQ:\n        min_indexes += [i]\n\nif(len(min_indexes) == nA):\n    return np.random.choice(actions)\nelse:  \n    for i in range(len(Qvals)):\n        if i in min_indexes:\n            probabilities[i] = (1 - eps) / len(min_indexes)\n        else:\n            probabilities[i] = eps / (nA - len(min_indexes))\n    return np.random.choice(actions, p=probabilities)\n   '

### 2. Model-based learning

You will now run the model-based learning algorithm discussed in class, and evaluate its learning performance.

---

#### Activity 3.        

Run the model-based reinforcement learning algorithm discussed in class to compute $Q^*$ for 100,000 iterations. Initialize each transition probability matrix as the identity and the cost function as all-zeros. Use an $\epsilon$-greedy policy with $\epsilon=0.1$ (use the function from Activity 2). Note that, at each step,

* You will need to select an action according to the $\epsilon$-greedy policy;
* The state and action, you will then compute the cost and generate the next state; 
* With this transition information (state, action, cost, next-state), you can now perform an update. 
* When updating the components $(x,a)$ of the model, use the step-size

$$\alpha_t=\frac{1}{N_t(x,a)+1},$$

where $N_t(x,a)$ is the number of visits to the pair $(x,a)$ up to time step $t$.

In order to ensure that your algorithm visits every state and action a sufficient number of times, after the boat reaches the goal cell, make one further step, the corresponding update, and then reset the position of the boat to a random state in the environment.

Plot the norm $\|Q^*-Q^{(k)}\|$ every 500 iterations of your method, where $Q^*$ is the optimal _Q_~function computed in Activity 1.

**Note:** The simulation may take a bit. Don't despair.

---

In [127]:
def apply_action(state, action):
    if action == 0:
        return [max(min(state[0] + 1 - WIND[state[1]], nrows -1), 0), state[1]]
    elif action == 1:
        return [max(min(state[0] - 1 - WIND[state[1]], nrows -1), 0), state[1]]
    elif action == 2:
        return [max(min(state[0] - WIND[state[1]], nrows -1), 0), max(min(state[1] - 1, ncols-1),0)]
    elif action == 3:
        return [max(min(state[0] - WIND[state[1]], nrows -1), 0), max(min(state[1] + 1, ncols-1),0)]

Q= np.zeros((nX, nA))
Pup = np.eye(nX)
Pdown = np.eye(nX)
Pleft = np.eye(nX)
Pright = np.eye(nX)

Prob = {0:Pup, 1:Pdown, 2:Pleft, 3:Pright}
cost = np.zeros((nX, nA))
visits = np.zeros((nX, nA))

state = init
for i in range(100000):
    action = epsilon_choice(Q, state)
    index = X.index(state)
    alpha = 1 / (visits[index][action] + 1)
    cost[index][action] = cost[index][action] + alpha * (c[index][action] - cost[index][action])
    new_state = apply_action(state, action)
    new_index = X.index(new_state)
    newP = Prob[action][index][new_index] + alpha * (1 - Prob[action][index][new_index])
    Prob[action][index] = Prob[action][index] + (alpha * (0 - Prob[action][index]))
    Prob[action][index][new_index] = newP
    visits[index][action] += 1
    Q[index][action] = cost[index][action] + Prob[action][index][new_index] * np.min(Q[new_index])
    state = next_state
print(Q)

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


### 3. Temporal-difference learning

You will now run both Q-learning and SARSA, and compare their learning performance with that of the model-based method just studied.

---

#### Activity 4.        

Repeat Activity 3 but using the _Q_-learning algorithm with a learning rate $\alpha=0.3$.

---

In [128]:
Ql = np.zeros((nX, nA))
alpha = 0.3
state = init
for i in range(100000):
    state_index = X.index(state)
    action = epsilon_choice(Q, state)
    next_state = apply_action(state, action)
    next_state_index  = X.index(next_state)
    Ql[state_index][action] = Ql[state_index][action] + (alpha * (c[state_index][action] + 
        (gamma * np.min(Q[next_state_index])) - Ql[state_index][action]))
    state = next_state
print(Ql)

[[ 0.          0.          0.          0.        ]
 [ 0.51        0.          0.          0.        ]
 [ 1.          0.83193     0.51        0.657     ]
 ..., 
 [ 0.          0.          0.          0.        ]
 [ 1.          1.          0.99999999  1.        ]
 [ 1.          1.          1.          1.        ]]


---

#### Activity 5.

Repeat Activity 4 but using the SARSA algorithm.

---

In [129]:
Ql = np.zeros((nX, nA))
alpha = 0.3
state = init
action = epsilon_choice(Q, state)
for i in range(100000):
    state_index = X.index(state)
    next_state = apply_action(state, action)
    next_state_index  = X.index(next_state)
    next_action = epsilon_choice(Q, next_state)
    Ql[state_index][action] = Ql[state_index][action] + (alpha * (c[state_index][action] + 
        (gamma * Q[next_state_index][next_action]) - Ql[state_index][action]))
    state = next_state
    action = next_action
print(Ql)

[[ 0.          0.          0.          0.        ]
 [ 0.3         0.          0.          0.        ]
 [ 1.          0.83193     0.3         0.51      ]
 ..., 
 [ 0.          0.          0.          0.        ]
 [ 1.          1.          0.99999999  1.        ]
 [ 1.          1.          1.          1.        ]]


---

#### Activity 6.

Discuss the differences observed between the performance of the three methods.

---

_Add your discussion here._